In [17]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler,RobustScaler
from geopy.distance import geodesic
import geopy.geocoders
from IPython.core.display import display, HTML
from datetime import datetime, timedelta

gl = geopy.geocoders.Nominatim(user_agent="geoapi")

fish_mapping = {
       'Indian Mackerel': 'Indian_Mackerel',
       'Rock Lobster': 'Rock_Lobster',
       'Silver Pomfret': 'Silver_Pomfret',
       'Indian White Prawn':'Indian_White_Prawn',
       'Spotted Crab':'Spotted_Crab',
       'Indo Pacific Seer Fish':'Indo_Pacific_Seer_Fish',
       'Johns Snapper':'Johns_Snapper'


    }
region={
    "Mohanpura Modern Fish Market": "Andaman and Nicobar",
        "Venlok Fish Market": "Andhra Pradesh",
        "Visakhapatnam Municipal Wholesale Fish Market": "Andhra Pradesh",
        "Ghoghla Retail Fish Market": "Daman and Diu",
        "Vanakbara Retail Fish Market": "Daman and Diu",
        "Mapusa Fish Market": "Goa",
        "SGPDA Wholesale Fish Market": "Goa",
        "Madikeri Retail Fish Market": "Karnataka",
        "Padubidri Retail Fish Market":"Karnataka",
        "Kavanad Retail Fish Market":"Kerala",
        "Ponnani Harbour Azheekkal":"Kerala",
        "Panvel Fish Market":"Maharashtra",
        "Sasoon Dock Retail Fish Market":"Maharashtra",
        "Naya Bazar Retail Fish Market":"Odisha",
        "Unit-4 Wholesale Fish Market":"Odisha",
        "Badagada Retail Fish Market":"Odisha",
        "Chintadripet Fish Market":"Tamil Nadu",
        "Kasivillangi Fish Market":"Tamil Nadu",
        "Nelpattai Fish Market":"Tamil Nadu"
     }

market_mapping = {
        "Mohanpura Modern Fish Market": "A",
        "Venlok Fish Market": "B",
        "Visakhapatnam Municipal Wholesale Fish Market": "C",
        "Ghoghla Retail Fish Market": "D",
        "Vanakbara Retail Fish Market": "E",
        "Mapusa Fish Market": "F",
        "SGPDA Wholesale Fish Market": "G",
        "Madikeri Retail Fish Market": "H",
        "Padubidri Retail Fish Market":"I",
        "Kavanad Retail Fish Market":"J",
        "Ponnani Harbour Azheekkal":"K",
        "Panvel Fish Market":"L",
        "Sasoon Dock Retail Fish Market":"M",
        "Naya Bazar Retail Fish Market":"N",
        "Unit-4 Wholesale Fish Market":"O",
        "Badagada Retail Fish Market":"P",
        "Chintadripet Fish Market":"Q",
        "Kasivillangi Fish Market":"R",
        "Nelpattai Fish Market":"S"

    }
market_locations = {
 "Mohanpura Modern Fish Market": (11.672381120028827, 92.7397),
        "Venlok Fish Market": (16.70938281077933, 81.1112177264719),
        "Visakhapatnam Municipal Wholesale Fish Market": (17.7053796532496, 83.303187),
        "Ghoghla Retail Fish Market": (20.714304955338925, 70.97391998412068),
        "Vanakbara Retail Fish Market":(20.719260442062694, 70.87390267725289),
        "Mapusa Fish Market": (15.28584289092577, 73.98999601587931),
        "SGPDA Wholesale Fish Market": (15.143243608628067, 74.10742815343389),
        "Madikeri Retail Fish Market": (15.143906409829622, 74.106999),
        "Padubidri Retail Fish Market":(13.145600326296021, 74.77176883068677),
        "Kavanad Retail Fish Market":(8.92457704338713, 76.55807833862643),
        "Ponnani Harbour Azheekkal":(10.787020188612708, 75.91824122805974),
        "Panvel Fish Market":(18.988859849961745, 73.11124035450575),
        "Sasoon Dock Retail Fish Market":(17.01878473985675, 73.29131601587935),
        "Naya Bazar Retail Fish Market":(20.447307041311962, 85.79736616931324),
        "Unit-4 Wholesale Fish Market":(20.2784362945832, 85.83048676176074),
        "Badagada Retail Fish Market":(20.293855508721297, 85.784482),
        "Chintadripet Fish Market":(13.075410427209148, 80.26967264279179),
        "Kasivillangi Fish Market":(10.835956010344615, 78.67219150793967),
        "Nelpattai Fish Market":(9.92221446292698, 78.12480551709594)

}
def get_distance(user_location, market_location):
    return geodesic(user_location, market_location).km

def get_coordinates(location_name):
    try:
        location = gl.geocode(location_name)
        return (location.latitude, location.longitude) if location else None
    except:
        return None

def make_clickable(val):
    return f'<a href="{val}" target="_blank">Route</a>' if "http" in val else val

def compare_markets(fore_date, fish, size, user_location):
    fish = fish_mapping.get(fish, fish)
    user_coords = get_coordinates(user_location)
    if not user_coords:
      return "Invalid location. Please enter a valid place."
    train_data = pd.read_excel("all_states_aggregate.xlsx")
    train_data.dropna(subset=['Date'], inplace=True)
    train_data['Date'] = pd.to_datetime(train_data['Date'], format='%d-%m-%Y', errors='coerce')
    full_dates = pd.DataFrame({'Date': pd.date_range(start=train_data['Date'].min(), end=train_data['Date'].max(), freq='D')})
    train_data = full_dates.merge(train_data, on='Date', how='left')
    train_data.set_index('Date', inplace=True)
    for col in train_data.columns:
        train_data[col] = pd.to_numeric(train_data[col], errors='coerce')
    train_data.replace(0, np.nan, inplace=True)
    # train_data = train_data.interpolate(method="polynomial", order=2, limit_direction="both")
    train_data = train_data.interpolate(method="linear", limit_direction="both")
    for col in train_data.columns:
        train_data[col] = train_data[col].fillna(train_data[col].median())
    for col in train_data.columns:
        if (train_data[col] < 0).any():
            smooth_mean = train_data.loc[train_data[col] > 0, col].mean()
            train_data[col] = train_data[col].apply(lambda x: x if x > 0 else smooth_mean)

    scaler = RobustScaler()
    train_scaled = pd.DataFrame(scaler.fit_transform(train_data),
                                columns=train_data.columns,
                                index=train_data.index)

    baseline_date = datetime.strptime("07-03-2025", "%d-%m-%Y")
    with open('model_ann.pkl', 'rb') as file:
        model = pickle.load(file)

    if fore_date < baseline_date:
        return "enter dates after 7th March,2025 only"

    steps = (fore_date - baseline_date).days

    input_data = train_scaled.values[-90:].reshape(1, 90, -1)
    forecast_scaled = []
    for _ in range(steps):
        pred = model.predict(input_data)
        forecast_scaled.append(pred[0])
        new_input = np.concatenate([input_data[0][1:], pred], axis=0)
        input_data = new_input.reshape(1, 90, -1)

    forecast = scaler.inverse_transform(forecast_scaled)
    forecast_dates = [baseline_date + timedelta(days=i + 1) for i in range(steps)]
    forecast_data = pd.DataFrame(forecast, columns=train_data.columns, index=forecast_dates)
    # fish_and_size = get_fish_column(fish, size, market)
    # if fish_and_size not in forecast_data.columns:
    #     return "Incorrect Input"
    # fish_data = forecast_data[fish_and_size]
    # prediction_value = np.round(forecast_data.loc[fore_date, fish_and_size])
    market_predictions = []
    for market, coords in market_locations.items():
        fish_and_size = f"{fish}_{size}_Market{market_mapping[market]}"
        if fish_and_size in forecast_data.columns:
            predicted_price = np.round(forecast_data.loc[fore_date, fish_and_size])
            distance = get_distance(user_coords, coords)
            maps_url = f"https://www.google.com/maps/dir/{user_coords[0]},{user_coords[1]}/{coords[0]},{coords[1]}"
            market_predictions.append({
                "Region": region[market],
                "Market": f"{market}",
                "Prediction in ₹": predicted_price,
                "Distance in km": round(distance, 2),
                "Route": maps_url
            })

        else:
            market_predictions.append({
                "Region": region[market],
                "Market": f"{market}",
                "Prediction in ₹": "Not Sold Here",
                "Distance in km": round(get_distance(user_coords, coords), 2),
                "Route": "N/A"
            })

    market_predictions.sort(key=lambda x: (x["Distance in km"] if isinstance(x["Distance in km"], (int, float)) else -1), reverse=False)


    df = pd.DataFrame(market_predictions)
    df["Route"] = df["Route"].apply(make_clickable)
    display(HTML(df.to_html(escape=False)))

In [18]:
compare_markets(datetime.strptime("07-04-2025","%d-%m-%Y"),"Indian Mackerel","Small","Hyderabad")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━

,Region,Market,Prediction in ₹,Distance in km,Route
0,Andhra Pradesh,Venlok Fish Market,Not Sold Here,289.87,N/A
1,Tamil Nadu,Chintadripet Fish Market,172.0,511.88,Route
2,Andhra Pradesh,Visakhapatnam Municipal Wholesale Fish Market,Not Sold Here,514.16,N/A
3,Goa,SGPDA Wholesale Fish Market,80.0,527.31,Route
4,Karnataka,Madikeri Retail Fish Market,255.0,527.32,Route
5,Goa,Mapusa Fish Market,245.0,531.30,Route
6,Maharashtra,Sasoon Dock Retail Fish Market,Not Sold Here,552.61,N/A
7,Maharashtra,Panvel Fish Market,245.0,595.28,Route
8,Karnataka,Padubidri Retail Fish Market,232.0,612.87,Route
9,Tamil Nadu,Kasivillangi Fish Market,130.0,722.21,Route
